# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902075


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:15,  2.70s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:49,  1.91s/it]

Rendering models:  23%|██▎       | 7/30 [00:05<00:31,  1.35s/it]

Rendering models:  30%|███       | 9/30 [00:07<00:25,  1.20s/it]

...

Rendering models:  77%|███████▋  | 23/30 [00:08<00:01,  5.81it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  6.78it/s]

Rendering models:  90%|█████████ | 27/30 [00:08<00:00,  8.13it/s]

Rendering models:  97%|█████████▋| 29/30 [00:08<00:00,  9.65it/s]

equidad1                              0.142927
kayleebug2017                         0.004187
JessieDawn                            0.001813
not-logged-in-c66140310d6a5859bd6c    0.004047
vehementscythe1337                    0.000855
Thedunkmasta                          0.003211
ElisabethB                            0.169865
FrancescoGullo                        0.002033
sinjinza1984                          0.001236
not-logged-in-6e654a118e00ea29ed67    0.030295
not-logged-in-22877e3a7af4aafb750a    0.002948
tingard                               0.000264
not-logged-in-4ddf669750977b537491    0.003696
Mckay_Oyler                           0.000346
TRISTANM18                            0.000356
acapirala                             0.005630
Lavadude                              0.005395
not-logged-in-15498ebaa693dd5a198d    0.000302
inhtih                                0.000704
ShirleyL                              0.000691
hunchoquavo                           6.129066
atayl233     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())